<a href="https://colab.research.google.com/github/ck1972/Geospatial-Deep-Learning/blob/main/Mod2_Lab1b_Introduction_to_DL_for_GeospatialDataAnalysis_CNN_EuroSat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1b.Deep Learning for Geospatial Data Analysis: Simple CNN on EuroSAT Dataset**


## Setup
### Install & import


In [ ]:
# Install & import libraries
# !pip install -q tensorflow matplotlib numpy tensorflow-datasets
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Define a simple CNN Model




In [ ]:
# Simple CNN Architecture (TensorFlow / Keras)
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

## Compile and train a simple CNN

In [ ]:
# Load and Preprocess EuroSAT Data
print("\nLoading EuroSAT dataset...")

dataset, info = tfds.load('eurosat', with_info=True, as_supervised=True)
train_ds, test_ds = dataset['train'], dataset['train'].take(500)  # use a subset for quick training

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (64, 64))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess).shuffle(1000).batch(32)
test_ds = test_ds.map(preprocess).batch(32)

# Train the CNN on EuroSAT
print("\nTraining model...")
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=5, validation_data=test_ds)

# Plot Accuracy
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Model Accuracy on EuroSAT")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

##  Visualize prediction

In [ ]:
#  Visualize pPredictions on EuroSAT
class_names = info.features['label'].names

plt.figure(figsize=(12, 12))
for images, labels in test_ds.take(1):
    predictions = model.predict(images)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        pred_label = np.argmax(predictions[i])
        true_label = labels[i].numpy()
        plt.title(f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}")
        plt.axis("off")
plt.suptitle("EuroSAT Model Predictions", fontsize=16)
plt.tight_layout()
plt.show()